# ATE Estimation

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os
from sklearn.linear_model import LogisticRegression

In [2]:
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

## Load and Format the Data

In [3]:
main_data = pd.read_csv("merged_with_hosts.csv")
main_data_cleaned = main_data.dropna(subset=["yes", "host_race_black", "host_gender_M", "multiple_listings", "shared_property", "ten_reviews", "log_price"])

In [4]:
main_data_cleaned.head()

,host_response,response_date,number_of_messages,automated_coding,latitude,longitude,bed_type,property_type,cancellation_policy,number_guests,...,baltimore,dallas,los_angeles,sl,dc,total_guests,raw_black,prop_black,any_black,past_guest_merge
0,1,2015-07-19 08:26:17,2.0,1,34.0815,-118.2700,Real Bed,House,Flexible,3.0,...,0,0,1,0,0,11.0,0.0,0.000000,0.0,matched (3)
1,0,2015-07-14 14:13:39,NaN,1,38.9107,-77.0198,NaN,House,Moderate,2.0,...,0,0,0,0,1,167.0,0.0,0.000000,0.0,matched (3)
2,2,2015-07-20 16:24:08,2.0,0,34.0047,-118.4810,Pull-out Sofa,Apartment,Strict,1.0,...,0,0,1,0,0,19.0,0.0,0.000000,0.0,matched (3)
3,10,2015-07-20 06:47:38,NaN,0,34.0917,-118.2820,NaN,House,Strict,8.0,...,0,0,1,0,0,41.0,0.0,0.000000,0.0,matched (3)
5,4,2015-07-18 18:07:19,NaN,0,34.0809,-118.3670,NaN,Apartment,Strict,3.0,...,0,0,1,0,0,263.0,1.0,0.003802,1.0,matched (3)


In [ ]:
# The same set confounders that the original paper's model includes
confounders = main_data_cleaned[["host_race_black", "host_gender_M", "multiple_listings", "shared_property", "ten_reviews", "log_price"]]
outcome = main_data_cleaned["yes"]
treatment = main_data_cleaned["guest_black"]

## Specify Nuisance Function Models

The next step is to specify models for the conditional expected outcome and propensity score

In [15]:
# specify a model for the conditional expected outcome

# make a function that returns a sklearn model for later use in k-folding
def make_Q_model():
  return RandomForestClassifier(n_estimators=100, max_depth=5)
Q_model = make_Q_model()

# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict_proba(X_test)[:,1]

test_ce=log_loss(y_test, y_pred)
print(f"Test CE of fit model {test_ce}") 
baseline_ce=log_loss(y_test, y_train.mean()*np.ones_like(y_test))
print(f"Test CE of no-covariate model {baseline_ce}")

Test CE of fit model 0.6676696953172495
Test CE of no-covariate model 0.6839255733272842


Because it is a randomized experiment. Treatment is randomly assigned and not confounded by X. Therefore, we can estimate the propensity score as g(x) = 0.5

## Use cross fitting to get get predicted outcomes and propensity scores for each unit

In [7]:
# helper functions to implement the cross fitting

def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    """
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"

    """
    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
      kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
      kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
      X_train = X_w_treatment.iloc[train_index]
      if not isinstance(y, pd.Series):
        y = pd.Series(y, index=X.index)
      y_train = y.iloc[train_index]
      q = make_model()
      q.fit(X_train, y_train)

      if output_type =='binary':
        predictions0[test_index] = q.predict_proba(X0.iloc[test_index])[:, 1]
        predictions1[test_index] = q.predict_proba(X1.iloc[test_index])[:, 1]
      elif output_type == 'continuous':
        predictions0[test_index] = q.predict(X0.iloc[test_index])
        predictions1[test_index] = q.predict(X1.iloc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [9]:
# Because it is a randomized experiment. Treatment is randomly assigned and not confounded by X. Therefore, we can estimate the propensity score as g(x) = 0.5
g = 0.5

In [151]:
Q0,Q1=outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, n_splits=10, output_type="binary")

In [152]:
data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})
data_and_nuisance_estimates.head()

,g,Q0,Q1,A,Y
0,0.5,0.416509,0.321200,0,1.0
1,0.5,0.602086,0.516874,0,0.0
2,0.5,0.571131,0.523646,0,0.0
3,0.5,0.475294,0.405418,1,0.0
5,0.5,0.548834,0.483744,0,1.0


## Combine predicted values and data into estimate of ATE

In [12]:
def ate_aiptw(Q0, Q1, g, A, Y, prob_t=None):
  """
  # Double ML estimator for the ATE
  """

  tau_hat = (Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g)).mean()
  
  scores = Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g) - tau_hat
  n = Y.shape[0] # number of observations
  std_hat = np.std(scores) / np.sqrt(n)

  return tau_hat, std_hat


In [154]:
tau_hat, std_hat = ate_aiptw(**data_and_nuisance_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is -0.08762009141834762 pm 0.024373805443655194


## Estimate ATE using a larger set of confounders

In the previous part, we used the same set of confounders as the original research. 
Now, we will use a larger set of confounders to estimate ATE.

In [22]:
# includes more host characteristics (did host have black guests before),
# listing characteristics (strict_cancellation, cleaning_fee, total number of guests),
# and neighborhood diversity (proportion of African Americans) as confounders
confounders = main_data_cleaned[["host_race_black", "host_gender_M", "multiple_listings",
                                  "shared_property", "ten_reviews", "log_price", "strict_cancellation",
                                  "has_cleaning_fee", "total_guests", "black_proportion", "any_black"]]

In [23]:
# Sanity check that chosen model actually improves test error
# A real analysis should give substantial attention to model selection and validation 

X_w_treatment = confounders.copy()
X_w_treatment["treatment"] = treatment

X_train, X_test, y_train, y_test = train_test_split(X_w_treatment, outcome, test_size=0.2)
Q_model.fit(X_train, y_train)
y_pred = Q_model.predict_proba(X_test)[:,1]

test_ce=log_loss(y_test, y_pred)
print(f"Test CE of fit model {test_ce}") 
baseline_ce=log_loss(y_test, y_train.mean()*np.ones_like(y_test))
print(f"Test CE of no-covariate model {baseline_ce}")

Test CE of fit model 0.6610129563360047
Test CE of no-covariate model 0.6922972921032778


In [24]:
Q0,Q1=outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, n_splits=10, output_type="binary")

In [21]:
g = 0.5

In [25]:
data_and_nuisance_estimates = pd.DataFrame({'g': g, 'Q0': Q0, 'Q1': Q1, 'A': treatment, 'Y': outcome})
data_and_nuisance_estimates.head()

,g,Q0,Q1,A,Y
0,0.5,0.497315,0.448491,0,1.0
1,0.5,0.657159,0.605675,0,0.0
2,0.5,0.516682,0.456506,0,0.0
3,0.5,0.506245,0.444406,1,0.0
5,0.5,0.637446,0.635651,0,1.0


In [26]:
tau_hat, std_hat = ate_aiptw(**data_and_nuisance_estimates)
print(f"The estimate is {tau_hat} pm {1.96*std_hat}")

The estimate is -0.08587845201896303 pm 0.02393858794799101


The outcome is similar to the original results